In [5]:
"""Heatmap."""

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from scipy.sparse import load_npz
from niceview.utils.tools import txt_to_list, select_col_from_name, normalize_array
from niceview.utils.tools import mask_to_image, blend

In [2]:
def heatmap():
    """Plot heatmap.

    Args:
        scores (np.ndarray): 1D array of scores of shape (N,). 
        coords (np.ndarray): 2D array of coordinates of shape (N, 2).
        patch_size (int): predefined size of the patch.
    """
    # image size in (width x, height y)
    # patch size
    # coordinates of the patch centers
    # normalize scores
    # heatmap overlay: tracks attention score over each pixel of heatmap
    # overlay counter: tracks number of times each pixel is visited
    # average heatmap: heatmap overlay / overlay counter (only for non-zero counter)
    # blur heatmap
    pass

In [6]:
import cv2
import numpy as np
from niceview.utils.tools import mask_to_image


def heatmap(
    dst_path,
    img_size,
    patch_size,
    coords,
    normalized_scores,
    cmap=cv2.COLORMAP_JET,
    overlap=-0.5,
):
    height, width = img_size
    patch_size = np.ceil(np.array(patch_size)).astype(int)
    overlay = np.full((height, width), 0).astype(float)
    counter = np.full((height, width), 0).astype(np.uint16) 
    for idx in range(len(coords)):
        score = normalized_scores[idx]
        coord = coords[idx]  # coord in (x, y)
        
        overlay[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += score
        counter[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += 1
    zero_mask = counter == 0
    overlay[~zero_mask] = overlay[~zero_mask] / counter[~zero_mask]
    kernel = tuple((patch_size * (1 - overlap)).astype(int) * 2 + 1)
    img = mask_to_image(overlay, cmap=cmap)
    blur = cv2.GaussianBlur(
        img, kernel, 0,
    )
    cv2.imwrite(dst_path, blur)

In [7]:
img_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'
cell_info_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-info.csv'
cell_gene_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene.npz'
cell_gene_name_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene-name.txt'
selected_cell_gene_name = 'ENSG00000065534'

In [10]:
dst_path = 'heatmap.png'
img = cv2.imread(img_path)
height, width, _ = img.shape
cell_info = pd.read_csv(cell_info_path)
cell_pos = cell_info[['x', 'y']].values.astype(int)
cell_gene = load_npz(cell_gene_path)
cell_gene_name = txt_to_list(cell_gene_name_path)
cell_selected_gene = select_col_from_name(
    cell_gene, cell_gene_name, selected_cell_gene_name,
)
cell_selected_gene_norm = normalize_array(cell_selected_gene, 1, 255)

In [12]:
heatmap(
    dst_path,
    (height, width),
    (64, 64),
    cell_pos,
    cell_selected_gene_norm,
)

In [ ]:
img_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'
cell_info_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-info.csv'
cell_gene_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene.npz'
cell_gene_name_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene-name.txt'
selected_cell_gene_name = 'ENSG00000065534'

In [4]:
img = cv2.imread(img_path)
height, width, _ = img.shape
base_size = (64, 64)
factor = 1
patch_size  = np.ceil(np.array(base_size)).astype(int) * factor

In [5]:
cell_info = pd.read_csv(cell_info_path)
cell_pos = cell_info[['x', 'y']].values
cell_gene = load_npz(cell_gene_path)
cell_gene_name = txt_to_list(cell_gene_name_path)
cell_selected_gene = select_col_from_name(
    cell_gene, cell_gene_name, selected_cell_gene_name,
)
cell_selected_gene_norm = normalize_array(cell_selected_gene, 1, 255)

In [6]:
cell_pos = cell_pos.astype(int)

In [7]:
overlay = np.full((height, width), 0).astype(float)
counter = np.full((height, width), 0).astype(np.uint16) 
for idx in range(len(cell_pos)):
    score = cell_selected_gene_norm[idx]
    coord = cell_pos[idx]  # coord in (x, y)
    
    overlay[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += score
    counter[coord[1]: coord[1] + patch_size[1], coord[0]: coord[0] + patch_size[0]] += 1

In [8]:
zero_mask = counter == 0

In [9]:
overlay[~zero_mask] = overlay[~zero_mask] / counter[~zero_mask]

In [10]:
overlap = -0.5
kernel = tuple((patch_size * (1 - overlap)).astype(int) * 2 + 1)
print(kernel)

(193, 193)


In [11]:
img = mask_to_image(overlay, cmap=cv2.COLORMAP_JET)

In [12]:
cv2.imwrite('gray.png', overlay)
cv2.imwrite('rgb.png', img)

True

In [13]:
blur = cv2.GaussianBlur(
    img, kernel, 0,
)

In [14]:
cv2.imwrite('blur.png', blur)

True

In [15]:
cv2.imwrite('blend.png', blend(img_path, 'blur.png', 0.5))

True

In [ ]:
# TODO: segmentation mask

In [ ]:
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter


def pathway_heatmap(image_shape, coords, color, path='heatmap.png'):
    """Plot a heatmap of the pathway.
    
    Args:
        image_shape (tuple): Shape of the image in ymax, xmax.
        coords (np.array): Coordinates of the pathway, in the form of (x, y).
        color (np.array): Color of the pathway.
        path (str): Path to the image.
    """
    xmax, ymax = image_shape
    
    ys = np.round(coords[:, 1]).astype(int)
    xs = np.round(coords[:, 0]).astype(int)
    zs = color
    
    num_intervals = 500
    xnew = np.linspace(xs.min(), xs.max(), num_intervals)
    ynew = np.linspace(ys.min(), ys.max(), num_intervals)
    xmesh, ymesh = np.meshgrid(xnew, ynew)
    zmesh = griddata((xs, ys), zs, (xmesh, ymesh), method='linear', rescale=True)
    smoothed_matrix = gaussian_filter(zmesh, 1)
    
    fig, ax = plt.subplots()
    fig.set_facecolor('black')
    ax.pcolormesh(xmesh, ymesh, smoothed_matrix, alpha=1, cmap='jet')
    ax.set_facecolor('black')
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_xlim(0, xmax)
    ax.set_ylim(ymax, 0)
    
    dpi = 2500
    fig.savefig(path, dpi=dpi, bbox_inches='tight', pad_inches=0)
    heatmap = cv2.imread(path)
    resized = cv2.resize(heatmap, (xmax, ymax), interpolation=cv2.INTER_AREA)
    cv2.imwrite(path, resized)